In [163]:
import csv 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json


In [164]:
df = pd.read_csv('.\\clean\\fire_incidents-csv.csv')
df.head()


C:\Users\judey\miniconda3\envs\jyEDA\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10,12,13,14,15,18,19,20,38,45,46,55,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point,longitude,latitude
0,"8,028,304",0,80283040,150 Elsie St.,2008-04-01T00:00:00,80920257,2008-04-01T18:06:37,2008-04-01T18:15:19,2008-04-01T18:21:48,SF,...,NaN,NaN,NaN,NaN,NaN,9.0,Bernal Heights,POINT (-122.41837339 37.74208979),-122.418373,37.742090
1,"8,028,303",0,80283030,85 Turner Tr.,2008-04-01T00:00:00,80920256,2008-04-01T18:00:52,2008-04-01T18:06:30,2008-04-01T18:22:18,SF,...,NaN,NaN,NaN,NaN,NaN,10.0,Potrero Hill,POINT (-122.39489 37.756291),-122.394890,37.756291
2,"8,028,309",0,80283090,175 6th St.,2008-04-01T00:00:00,80920262,2008-04-01T18:42:06,2008-04-01T18:45:23,2008-04-01T18:53:25,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,South of Market,POINT (-122.407468 37.78008),-122.407468,37.780080
3,"8,028,314",0,80283140,633 Hayes St.,2008-04-01T00:00:00,80920268,2008-04-01T19:03:52,2008-04-01T19:08:39,2008-04-01T19:35:36,SF,...,NaN,NaN,NaN,NaN,NaN,5.0,Hayes Valley,POINT (-122.42684908 37.77612642),-122.426849,37.776126
4,"8,028,319",0,80283190,27th Av. / Cabrillo St.,2008-04-01T00:00:00,80920273,2008-04-01T19:16:12,2008-04-01T19:23:48,2008-04-01T19:28:49,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,Outer Richmond,POINT (-122.4863941 37.77428492),-122.486394,37.774285


In [165]:
df["Primary Situation"] = df["Primary Situation"].str.replace(" - ", " ")
print(len(df["Primary Situation"].unique()))
pd.DataFrame(df["Primary Situation"].unique()).to_csv('.\\clean\\primarysituation.csv')

257


In [166]:
df["Primary Situation"] = df["Primary Situation"].str.split(' ').str[0]
print(len(df["Primary Situation"].unique()))
df["Primary Situation"].unique()

188


array(['412', '552', '210', '522', '520', '733', '711', '113', '743',
       '551', '411', '150', '413', '745', '118', '442', '322', '735',
       '554', '311', '700', '600', '142', '111', '550', '444', '351',
       '331', '500', '744', '653', '900', '323', '451', '445', '511',
       '651', '730', '131', '531', '740', '741', '553', '151', '100',
       '710', '140', '463', '220', '715', '130', '154', '223', '713',
       '721', '300', '712', '321', '352', '714', '353', '400', '114',
       '911', '731', '410', '440', '460', '652', '160', '221', '521',
       '650', '422', '812', '621', '240', '672', '510', '213', '611',
       '360', '542', '661', '671', '461', '200', '441', '541', '443',
       '381', '641', '364', '736', '555', '742', '350', '11', '356',
       '423', '112', '361', '561', '632', '362', '420', '746', '431',
       '340', '251', '482', '365', '372', '371', '540', '734', '153',
       '163', '512', '462', '370', '122', '152', '123', '143', '212',
       '421', '164', 

In [167]:
primarySituationCounts = df.groupby("Primary Situation").count()["Incident Number"]
pd.DataFrame(primarySituationCounts.unique()).to_csv('.\\clean\\primarysituation.csv')


In [168]:
top10_situations = primarySituationCounts.sort_values(ascending=False).head(10)
print(top10_situations)

Primary Situation
711    67185
700    50772
745    31652
735    26023
500    19986
322    19928
113    18435
311    18256
743    16900
733    14835
Name: Incident Number, dtype: int64


At this point I took the primary situation codes dumped to CSV and then cleaned them with OpenRefine, then exported them to primarysituation-csv.csv. Then, using this file as a lookup, I iterate through the top 10 primary situations, get the proper name value from this CSV, and dump that to JSON, from there its visualized with PowerBI. 

In [170]:
primarySituationCodes = pd.read_csv('.\\clean\\primarysituation-csv.csv')

def get_sitatuation_Name_for_code(code):
    return primarySituationCodes[primarySituationCodes["Code"] == code].values[0][0]




In [171]:
key_values = dict()
for key in top10_situations.keys():
    key_values[get_sitatuation_Name_for_code(key)] = str(top10_situations[key])

with open('.\\clean\\topX_primarySituations.json', 'w') as convert_file:
     convert_file.write(json.dumps(key_values))